
## titanic competition

- 이번 프로젝트에서는 간단한 분류 문제를 풀어봅니다.

- sklearn으로 머신러닝 모델을 구현해봅니다.

- Machine Learning Workflow를 따라가봅니다.


Source : https://www.kaggle.com/c/titanic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
# titanic data 불러오기
import pandas as pd

base_path= "C:/Users/gmlkd/machine-learning-exercises/data/titanic/"

train = pd.read_csv(base_path+ "train.csv")
test= pd.read_csv(base_path+ "test.csv")
submission = pd.read_csv(base_path+ "gender_submission.csv")

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Data Preprocessing

1. 결측치 처리


2. feature selection (분석에 사용하지 않을 column 제거)

In [11]:
# titanic data에서 missing value를 찾아봅니다.
train[train.isnull().any(axis=1)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [22]:
# Embarked column이 NaN인 row를 찾습니다.
# 둘 다 여성이며 ticket번호, Pclass가 같다
train[train.Embarked.isnull()]

# 최빈값인 S로 채울 수 있을 것 같다
# train.Embarked.value_counts() 

# pclass=1, sex=female인 사람들의 embarked
train[(train.Pclass==1) & (train.Sex=='female')]['Embarked'].value_counts()

# null값 채우기
train.loc[train.Embarked.isnull(), "Embarked"] = "S"
train[(train.Pclass==1) & (train.Sex=='female')]['Embarked'].value_counts()

S    50
C    43
Q     1
Name: Embarked, dtype: int64

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
# missing value를 handling 합니다.
# column을 지울까요 / 채울까요?

# drop해도 될 것 같다
# train.Cabin.value_counts()

# +추가로 drop해도 될 column
# PassengerId, Name, Ticket, Cabin
train = train.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [30]:
# "Age" column 채우기 (평균으로)
train = train.fillna(train.Age.mean())
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


### Feature Engineering

1. Categorical feature encoding

2. Normalization

In [32]:
# categorical feature --> One-hot Encoding / Ordinal Encoding

# 1. Ordinal Encoding -> Ordinal feature를 변활할 때 쓰임 eg) 학력, 선호도
# 2. One-hot Encoding -> Nominal feature를 변환할 때 쓰임 eg) 성별, 부서, 출신학교

# One-hot Encoding, 하나의 정보만 1로 만들어준다.
train_OHE = pd.get_dummies(train, columns=['Sex','Embarked'])
train_OHE

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,1,0,0


In [51]:
# Normalization --> Min-Max scaling

X = train_OHE.drop(columns='Survived') #input matrix
y = train_OHE.Survived                 # target vector

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)

# 두개의 컬럼만 스케일링 할 때
temp = scaler.fit_transform(X.loc[:, ['Age','Fare']]) 
X['Age'] = temp[:, 0]
X['Fare'] = temp[:, 1]
X


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,0.271174,1,0,0.014151,0,1,0,0,1
1,1,0.472229,1,0,0.139136,1,0,1,0,0
2,3,0.321438,0,0,0.015469,1,0,0,0,1
3,1,0.434531,1,0,0.103644,1,0,0,0,1
4,3,0.434531,0,0,0.015713,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,2,0.334004,0,0,0.025374,0,1,0,0,1
887,1,0.233476,0,0,0.058556,1,0,0,0,1
888,3,0.367921,1,2,0.045771,1,0,0,0,1
889,1,0.321438,0,0,0.058556,0,1,1,0,0


### Training 

In [37]:
# sklearn에서 배웠던 분류 모델들을 불러와봅니다.
#1. Linear Classifier
from sklearn.linear_model import SGDClassifier
#2. Logistic Regression
from sklearn.linear_model import LogisticRegression
#3. Decision Tree
from sklearn.tree import DecisionTreeClassifier
#4. Random Forest
from sklearn.ensemble import RandomForestClassifier

#평가 지표
from sklearn.metrics import accuracy_score

In [52]:
clf = SGDClassifier()
clf2 = LogisticRegression(max_iter=5000)
clf3 = DecisionTreeClassifier()
clf4 = RandomForestClassifier()

clf.fit(X, y)
clf2.fit(X,y)
clf3.fit(X,y)
clf4.fit(X,y)

# 학습에 대한 성능평가 (예측 성능x)
pred = clf.predict(X)
pred2 = clf2.predict(X)
pred3 = clf3.predict(X)
pred4 = clf4.predict(X)

In [53]:
print("1. Linear Classifier, Accuracy: %.4f " % accuracy_score(y, pred))
print("2. LogisticRegreesion, Accuracy: %.4f " % accuracy_score(y, pred2))
print("3. DecisionTree, Accuracy: %.4f " % accuracy_score(y, pred3))
print("4. RandomForest, Accuracy: %.4f " % accuracy_score(y, pred4))

1. Linear Classifier, Accuracy: 0.7755 
2. LogisticRegreesion, Accuracy: 0.8013 
3. DecisionTree, Accuracy: 0.9820 
4. RandomForest, Accuracy: 0.9820 


### Test (Predict)

In [44]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [46]:
# test data에 같은 feature engineering을 적용해줍니다.
test = test.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [59]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [60]:
# 중요! train데이터로 처리했기에 전처리도 train기준으로
test = test.fillna(train.Age.mean()) 
test = test.fillna(train.Fare.mean()) 

In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [62]:
# Categorical feature encoding
test_OHE = pd.get_dummies(data=test, columns=['Sex','Embarked'])
test_OHE

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.500000,0,0,7.8292,0,1,0,1,0
1,3,47.000000,1,0,7.0000,1,0,0,0,1
2,2,62.000000,0,0,9.6875,0,1,0,1,0
3,3,27.000000,0,0,8.6625,0,1,0,0,1
4,3,22.000000,1,1,12.2875,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,3,29.699118,0,0,8.0500,0,1,0,0,1
414,1,39.000000,0,0,108.9000,1,0,1,0,0
415,3,38.500000,0,0,7.2500,0,1,0,0,1
416,3,29.699118,0,0,8.0500,0,1,0,0,1


In [63]:
# Normalization
temp = scaler.transform(test_OHE.loc[:, ['Age','Fare']])
test_OHE.Age = temp[:,0]
test_OHE.Fare = temp[:,1]

In [64]:
test_OHE.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,0.428248,0,0,0.015282,0,1,0,1,0
1,3,0.585323,1,0,0.013663,1,0,0,0,1
2,2,0.773813,0,0,0.018909,0,1,0,1,0
3,3,0.334004,0,0,0.016908,0,1,0,0,1
4,3,0.271174,1,1,0.023984,1,0,0,0,1


In [65]:
# fit은 test data에 y값이 없으므로 할 수 없다.

# predict
result = clf.predict(test_OHE)
result2 = clf2.predict(test_OHE)
result3 = clf3.predict(test_OHE)
result4 = clf4.predict(test_OHE)

In [67]:
# 결과 파일인 submission.csv를 생성합니다.
submission['Survived'] = result4
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


- 모든 학습이 끝나면 결과를 가지고 캐글에 제출해볼 수 있습니다.

https://www.kaggle.com/c/titanic

In [68]:
submission.to_csv('submission.csv', index=False)